# Exctract Historic Market Cap data
* [Coingecko python code](https://github.com/man-c/pycoingecko)

In [32]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

## Initialize classes

In [42]:
import os
import sys
import configparser
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from wrangler.modules.assets.etl import cryptoMCExtractor as crypto
# from utils.modules.etl.load import noSQLwls as nosql
from utils.modules.etl.load import sparkNoSQLwls as nosql

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    crypto = importlib.reload(crypto)
    nosql = importlib.reload(nosql)

clsNoSQL = nosql.NoSQLWorkLoads(desc="get crypto macket capitalization data")
''' optional - if not specified class will use the default values '''
prop_kwargs = {"WRITE_TO_TMP":True,   # necessary to emulate the etl dag
              }
clsMC = crypto.CryptoMarkets(
    desc='get crypto macket capitalization data', **prop_kwargs)
print("\nClass initialization and load complete!")

All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All assets-module etl-packages in function-CryptoMarket imported successfully!
All functional SPARKNOSQLWLS-libraries in LOAD-package of ETL-module imported successfully!
sparkNoSQLwls Class initialization complete
sparkNoSQLwls Class initialization complete
CryptoMarket Class initialization complete

Class initialization and load complete!


## Read catalog data
* Distinct API categories: "asset.metadata", "historic.prices", "latest.prices"

In [41]:
from datetime import datetime
from bson.json_util import dumps

__now_dt__ = str(datetime.utcnow().isoformat())
__as_type__ = 'dict'
__database__ = "tip-asset-metadata"
__collection__ = "coingecko.asset.list"
__find__ = {"asset.id":{"$in":['arenaplay']},
             "source":{"$eq":"coingecko"},
           }

clsNoSQL.connect={'DBAUTHSOURCE':__database__}
_data = clsNoSQL.read_documents(
    as_type=__as_type__,
    db_name = __database__,
    db_coll=__collection__,
    doc_find=__find__
)
print(dumps(_data, indent=2))

[
  {
    "_id": {
      "$oid": "638740c7d8ab1a48899beddc"
    },
    "source": "coingecko",
    "name": "ArenaPlay",
    "symbol": "apc",
    "lastupdated": {
      "$date": "2022-12-01T12:12:24.350Z"
    },
    "asset.id": "arenaplay",
    "asset.isactive": null,
    "asset.tokenaddress": null,
    "asset.platforms": {
      "binance-smart-chain": "0x2aa504586d6cab3c59fa629f74c586d78b93a025"
    },
    "asset": {
      "id": "arenaplay",
      "isactive": -1.0,
      "platforms": null,
      "tokenaddress": null,
      "type": "altcoin"
    }
  }
]


## Extract Historic Marketcap

In [43]:
import time

_data_owner='coingecko'
# kwargs = {}

kwargs = {"ASSETS":[
    {
    "_id": {
      "$oid": "638740c7d8ab1a48899bf0a5"
    },
    "source": "coingecko",
    "name": "Bitcoin",
    "symbol": "btc",
    "lastupdated": {
      "$date": "2022-11-30T19:43:19.855Z"
    },
    "asset.id": "bitcoin",
    "asset.isactive": 0,
    "asset.tokenaddress": '',
    "asset.platforms": {},
    "asset": {
      "id": "bitcoin",
      "isactive": 1.0,
      "platforms": {},
      "tokenaddress": ''
    }
  }]
         }

_from_dt=date(2022,8,1)
_to_dt=date(2022,8,31)

_results=None
_results, _failed = clsMC.extract_historic_mcap(
    data_owner=_data_owner,
    from_date = _from_dt,
    to_date = _to_dt,
    **kwargs,
)
if _results is None:
    print("No results but process completed")
else:
    print("Process completed with %d historic extractions" %len(_results))

Process completed with 1 historic extractions


In [35]:
import re
logpath = os.path.join("/home/nuwan/workspace/rezaware/",
                     "wrangler/logs/assets/etl/app.log")
with open(logpath,'r') as f:
    symbol_list=[]
    for line in f:
        if line.startswith("[WARNING]"):
            m = re.search("etl - (.+?) marketcap", line)
            if m:
                found = m.group(1)
                symbol_list.append(found.replace("'",""))
print(symbol_list)

['arenaplay', 'argo-finance', 'argonon-helium', 'bacon-coin', 'balloonsville-air', 'beta-token', 'bitcoin-asset-2', 'bitcoin-free-cash', 'bitmon', 'block-forest', 'bns-token', 'bombcrypto-coin', 'bone-2', 'bounty', 'build', 'bullet-2', 'b-watch-box', 'capricorn', 'chi-coin', 'connect-token', 'crossx', 'cryptounit', 'crypto-vault', 'd-drops', 'dracoomaster', 'dragonmaster-token', 'everstart', 'galaxy-finance', 'giresunspor-token', 'graviton', 'greenhouse', 'landboard', 'memeverse', 'moonrock-v2', 'scholarship-coin', 'stepwatch', 'trivian', 'visiongame']


In [35]:
import json

logpath = os.path.join("/home/nuwan/workspace/rezaware/",
                     "wrangler/notebooks/assets/etl/crypto/nft_list.json")
nft_dict=[]
with open(logpath,'r') as f:
    nft_list = json.load(f)
nft_symb_list = [x['symbol'] for x in nft_list]
print(nft_symb_list)

['KONGZ', 'MZ', 'SYL', 'GAIA', 'SYL', 'BOMUL', 'MIYA', 'SYL', 'SW', 'GAIA', 'BELLYGOM', 'SNKRZ', 'etnah', 'MTDZ', 'DOD', 'Puuvilla', 'GAIA', 'SWC', 'kongz', 'MATE', 'K3K', 'COGO', 'UKKIKKI', 'ArcheWorld_Land', 'SMILEMIYA', 'KDGE', 'VGN', 'TREASURES', 'ArcheWorld_FandomCard', 'SE7EN', 'HT', 'MTM', 'DHB', 'RBL', 'CELPI', 'GAIA', 'GRA', 'chikn', 'BIRDIE CC', 'NRS', 'KROCK', 'LCV1', 'MTG', 'APB', 'BUMZ', 'AOZHEROES', 'KLND', 'TXP', 'FOOTAGE', 'LPV1', 'KLTW', 'UKIDS', 'CT', 'PXL', 'LEMONG 3D', 'BIRDIE', 'bayc', 'SGS', 'PUNK', 'TPC', 'larva', 'KING-KINGS', 'SNKRZ OPEN BOX', '7', 'Puuvillafriends', 'NAC', 'Born Kongz', 'WTBL', 'BK', 'DMONG', 'MEE', 'LIITA', 'DUK', 'KENOME', 'KRBC', 'ROAZ', 'EMATES', 'MAC', 'KLAPES', 'MTDZPET', 'CYECV0', 'EV', 'PG', 'psd', 'CATTP', 'Roostr', 'DHBSRP', 'TMH', 'HDEC75', 'GTU', 'SLIME', 'MAYC', 'MT', 'CMPZPFP', 'RDK', 'ALAP', 'STORKS', 'KONGZ', 'BUMZ', 'MA']


In [49]:
clsNoSQL.dbType = 'mongodb'
clsNoSQL.dbName = 'tip-historic-marketcap'
clsNoSQL.collections = {"HASINNAME":'btc'}
print(clsNoSQL.collections)

['coingecko.2022-01-01.wbtc', 'coingecko.2022-07-01.obt', 'coingecko.2022-07-01.btcmt', 'coingecko.2022-04-01.nsbt', 'coingecko.2022-01-01.nsbt', 'coingecko.2022-07-01.btc2', 'coingecko.2022-01-01.bto', 'coingecko.2022-07-01.ibbtc', 'coingecko.2022-01-01.btcred', 'coingecko.2022-01-01.btse', 'coingecko.2022-01-01.btcz', 'coingecko.2022-07-01.btrfly', 'coingecko.2022-01-01.btc', 'coingecko.2022-04-01.btg', 'coingecko.2022-04-01.renbtc', 'coingecko.2022-01-01.btu', 'coingecko.2022-04-01.btx', 'coingecko.2022-04-01.btl', 'coingecko.2022-07-01.wbtc', 'coingecko.2022-04-01.obtc', 'coingecko.2022-07-01.btnt', 'coingecko.2022-07-01.xbtx', 'coingecko.2022-01-01.btcp', 'coingecko.2022-04-01.wbtc', 'coingecko.2022-01-01.hbtc', 'coingecko.2022-07-01.btc', 'coingecko.2022-04-01.xbtx', 'coingecko.2022-01-01.hbt', 'coingecko.2022-04-01.0xbtc', 'coingecko.2022-07-01.btc2x-fli', 'coingecko.2022-01-01.obt', 'coingecko.2022-01-01.yfbt', 'coingecko.2022-07-01.btb', 'coingecko.2022-07-01.btcst', 'coingeck